In [0]:
!unzip ef8eb660168111ea.zip

Archive:  ef8eb660168111ea.zip
   creating: Dataset/
  inflating: Dataset/Train.csv       
  inflating: Dataset/sample_submission.csv  
  inflating: Dataset/Test.csv        


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv("Dataset/Train.csv")

In [0]:
data.head()

,id,realtionship_status,industry,genre,targeted_sex,average_runtime(minutes_per_week),airtime,airlocation,ratings,expensive,money_back_guarantee,netgain
0,19717,Married-spouse-absent,Auto,Comedy,Male,45,Primetime,United-States,0.027465,High,No,False
1,31593,Married-civ-spouse,Pharma,Comedy,Male,45,Primetime,United-States,0.027465,Low,No,False
2,5681,Divorced,Entertainment,Comedy,Female,45,Primetime,United-States,0.027465,High,Yes,False
3,15491,Separated,Political,Infomercial,Female,40,Primetime,United-States,0.027465,Low,No,False
4,23587,Married-civ-spouse,Pharma,Comedy,Male,48,Primetime,United-States,0.027465,High,No,True


In [0]:
relationship_status = dict((w,i) for i,w in enumerate(set(data.realtionship_status)))
industry = dict((w,i) for i,w in enumerate(set(data.industry)))
genre = dict((w,i) for i,w in enumerate(set(data.genre)))
targeted_sex = dict((w,i) for i,w in enumerate(set(data.targeted_sex)))
airtime = dict((w,i) for i,w in enumerate(set(data.airtime)))
airlocation = dict((w,i) for i,w in enumerate(set(data.airlocation)))
expensive = dict((w,i) for i,w in enumerate(set(data.expensive)))
money_back_gurantee = dict((w,i) for i,w in enumerate(set(data.money_back_guarantee)))
netgain = dict((w,i) for i,w in enumerate(set(data.netgain)))
netgain_rev = dict((i,w) for i,w in enumerate(set(data.netgain)))

In [0]:
data.realtionship_status = data.realtionship_status.map(relationship_status)
data.industry = data.industry.map(industry)
data.genre = data.genre.map(genre)
data.targeted_sex = data.targeted_sex.map(targeted_sex)
data.airtime = data.airtime.map(airtime)
data.airlocation = data.airlocation.map(airlocation)
data.expensive = data.expensive.map(expensive)
data.money_back_guarantee = data.money_back_guarantee.map(money_back_gurantee)
data.netgain = data.netgain.map(netgain)

In [0]:
data

,id,realtionship_status,industry,genre,targeted_sex,average_runtime(minutes_per_week),airtime,airlocation,ratings,expensive,money_back_guarantee,netgain
0,19717,3,5,3,1,45,1,30,0.027465,2,1,0
1,31593,1,3,3,1,45,1,30,0.027465,0,1,0
2,5681,0,4,3,0,45,1,30,0.027465,2,0,0
3,15491,6,1,2,0,40,1,30,0.027465,0,1,0
4,23587,1,3,3,1,48,1,30,0.027465,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
26043,16009,1,3,3,1,50,1,30,0.027465,0,1,1
26044,17241,4,5,3,0,40,2,30,0.027465,0,1,0
26045,2295,1,0,3,0,25,1,30,0.102234,0,1,1
26046,17902,4,5,3,0,48,2,30,0.027465,0,0,0


In [0]:
data.corr()

,id,realtionship_status,industry,genre,targeted_sex,average_runtime(minutes_per_week),airtime,airlocation,ratings,expensive,money_back_guarantee,netgain
id,1.000000,0.003466,0.004626,0.008772,0.001849,0.004017,0.001865,0.005257,0.003047,0.001003,0.009900,0.006263
realtionship_status,0.003466,1.000000,0.040867,-0.087910,-0.210680,-0.196047,0.352989,-0.020082,-0.062806,0.011763,-0.001160,-0.288184
industry,0.004626,0.040867,1.000000,0.014575,0.023142,0.153033,-0.201771,0.007763,-0.005674,-0.003992,0.000599,-0.077961
genre,0.008772,-0.087910,0.014575,1.000000,0.079723,0.031436,-0.024830,0.165698,-0.003336,0.002835,-0.006143,0.049647
targeted_sex,0.001849,-0.210680,0.023142,0.079723,1.000000,0.217584,-0.082231,0.002058,0.043120,-0.008241,0.003237,0.202949
average_runtime(minutes_per_week),0.004017,-0.196047,0.153033,0.031436,0.217584,1.000000,-0.068833,0.009678,0.080649,0.008236,0.007487,0.204205
airtime,0.001865,0.352989,-0.201771,-0.024830,-0.082231,-0.068833,1.000000,0.002027,-0.076505,0.008837,0.008579,-0.236727
airlocation,0.005257,-0.020082,0.007763,0.165698,0.002058,0.009678,0.002027,1.000000,-0.004088,-0.007561,-0.000966,0.022306
ratings,0.003047,-0.062806,-0.005674,-0.003336,0.043120,0.080649,-0.076505,-0.004088,1.000000,-0.001202,-0.003504,0.229677
expensive,0.001003,0.011763,-0.003992,0.002835,-0.008241,0.008236,0.008837,-0.007561,-0.001202,1.000000,0.257406,0.008089


In [0]:
from sklearn.linear_model import LinearRegression , LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier ,XGBRegressor

In [0]:
x = data.drop(["id","netgain"],axis=1)
y= data[["netgain"]]

In [0]:
# from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# encoded_x = None
# for i in range(0, x.shape[1]):
# 	label_encoder = LabelEncoder()
# 	feature = label_encoder.fit_transform(x.values[:,i])
# 	feature = feature.reshape(x.shape[0], 1)
# 	onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
# 	feature = onehot_encoder.fit_transform(feature)
# 	if encoded_x is None:
# 		encoded_x = feature
# 	else:
# 		encoded_x = np.concatenate((encoded_x, feature), axis=1)
  
# label_encoded_y = LabelEncoder().fit_transform(y)

In [0]:
scale = MinMaxScaler()
x = scale.fit_transform(x)
x_train,x_test , y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=125)

In [0]:
testing = pd.read_csv("Dataset/Test.csv")
testing.realtionship_status = testing.realtionship_status.map(relationship_status)
testing.industry = testing.industry.map(industry)
testing.genre = testing.genre.map(genre)
testing.targeted_sex = testing.targeted_sex.map(targeted_sex)
testing.airtime = testing.airtime.map(airtime)
testing.airlocation = testing.airlocation.map(airlocation)
testing.expensive = testing.expensive.map(expensive)
testing.money_back_guarantee = testing.money_back_guarantee.map(money_back_gurantee)

In [0]:
try1=LogisticRegression(
    penalty="l1",
    tol=0.1
)
try1.fit(x_train,y_train)
print(try1.score(x_test,y_test))

import numpy as np
np.unique(try1.predict(scale.transform(testing.drop(["id"],axis=1))),return_counts=True)

0.7919651995905834


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(array([0, 1]), array([5976,  537]))

In [0]:
x_train

array([[0.16666667, 0.        , 0.75      , ..., 0.02746467, 1.        ,
        1.        ],
       [0.16666667, 0.6       , 0.75      , ..., 0.02746467, 0.        ,
        0.        ],
       [0.16666667, 0.        , 0.75      , ..., 0.02746467, 1.        ,
        0.        ],
       ...,
       [0.66666667, 0.8       , 0.5       , ..., 0.02746467, 1.        ,
        1.        ],
       [0.16666667, 0.6       , 0.75      , ..., 0.02746467, 0.        ,
        1.        ],
       [0.16666667, 0.6       , 0.75      , ..., 0.02746467, 0.        ,
        0.        ]])

In [0]:
!pip install catboost

     |████████████████████████████████| 63.9MB 47kB/s 


In [0]:
x_test

array([[1.        , 0.8       , 0.5       , ..., 0.02746467, 0.        ,
        0.        ],
       [0.66666667, 1.        , 0.75      , ..., 0.02746467, 1.        ,
        1.        ],
       [0.        , 1.        , 0.75      , ..., 0.02746467, 0.5       ,
        1.        ],
       ...,
       [0.16666667, 0.6       , 0.75      , ..., 0.02746467, 1.        ,
        1.        ],
       [0.66666667, 1.        , 0.75      , ..., 0.02746467, 0.        ,
        0.        ],
       [0.66666667, 0.2       , 0.75      , ..., 0.02746467, 1.        ,
        1.        ]])

In [0]:
from catboost import CatBoostClassifier

param_catboost={
    "learning_rate":[0.3,0.4],
    "depth":[1,2,3],
    "n_estimators":[800,900,1000],
    "loss_function":["CrossEntropy"],
    "reg_lambda":[0.1,0.2,0.3],
    "eval_metric":["Accuracy"],
    "verbose":[False],
    "task_type":["GPU"]
 }


from sklearn.model_selection import RandomizedSearchCV , GridSearchCV
cat = CatBoostClassifier()
random = GridSearchCV(cat,param_grid=param_catboost,cv=5,verbose=True,n_jobs=2)
random.fit(x,y)
# random.score(x_test,y_test)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 19.0min
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed: 83.4min
[Parallel(n_jobs=2)]: Done 270 out of 270 | elapsed: 117.0min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=<catboost.core.CatBoostClassifier object at 0x7f098fef6e80>,
             iid='warn', n_jobs=2,
             param_grid={'depth': [1, 2, 3], 'eval_metric': ['Accuracy'],
                         'learning_rate': [0.3, 0.4],
                         'loss_function': ['CrossEntropy'],
                         'n_estimators': [800, 900, 1000],
                         'reg_lambda': [0.1, 0.2, 0.3], 'task_type': ['GPU'],
                         'verbose': [False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [0]:
test(random)

(array([0., 1.]), array([5436, 1077]))


In [0]:
def test(model):
  import numpy as np

  test=testing.drop(["id"],axis=1)
  test=scale.transform(test)
  al = model.predict(test)
  print(np.unique(model.predict(test),return_counts=True))
  save_result(al)

In [0]:
def save_result(al):
  pd.DataFrame(data=list(zip(testing.id,al)),columns=["id","netgain"],index=None).to_csv("submission.csv",index=None)

In [0]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(
    n_estimators=100
)
model.fit(x_train,y_train)
print(model.score(x_test,y_test))
import numpy as np

test=testing.drop(["id"],axis=1)
test=scale.transform(test)
al = model.predict(test)
np.unique(model.predict(test),return_counts=True)
save_result(al)

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

model = SVC(kernel="linear")
model.fit(x_train,y_train)
print(accuracy_score(model.predict(x_test),y_test))
test=testing.drop(["id"],axis=1)
test=scale.transform(test)
al = model.predict(test)
np.unique(model.predict(test),return_counts=True)
save_result(al)

In [0]:
import time 
start = time.time()
model = XGBClassifier(
    n_estimators=1200,
    num_leaves=250,
    max_depth=2,
    learning_rate=0.1,
    gamma=0.01,
    min_child_weight=0.001,
    reg_alpha=0.2,
    reg_lambda=0.2,
)
model.fit(x_train,y_train)
end = time.time()
print("completed in ",(end-start)/60," minutes")
print(model.score(x_test,y_test))

al =[]

# test = testing[x_train.columns]
# print(test.columns)
for i,row in testing.drop(["id"],axis=1).iterrows():
  row = scale.transform([row])
  al.append(model.predict(row)[0])

import numpy as np
np.unique(al,return_counts=True)
save_result(al)

In [0]:
b = x_test[x_train.columns]

In [0]:
params_lgbm = {
    'max_depth': [0,1,2,3,4,5,7,8],
    'min_child_samples': [15,18,20,25,30,35],
    'min_child_weight': [0.01,0.05,0.001],
    'min_split_gain': [0.001,0.01,0.0],
    'n_estimators': [240,250,300,320,330,350,400,450,500],
    'num_leaves': [150,170,180,200,220,250,300,350],
    'reg_alpha': [0,0.1,0.2,0.25],
    'reg_lambda': [0,0.1,0.2,0.25],
    'subsample_freq': [1,2,4,5,6,7,8]
 }

params_xgboost = {
    'max_depth': [0,1,2,3,4,5,7,8],
    'min_child_samples': [15,18,20,25,30,35],
    'min_child_weight': [0.01,0.05,0.001],
    'min_split_gain': [0.001,0.01,0.0],
    'n_estimators': [240,250,300,320,330,350,400,450,500],
    'num_leaves': [150,170,180,200,220,250,300,350],
    'reg_alpha': [0,0.1,0.2,0.25],
    'reg_lambda': [1,0.9,0.95,0.85],
    'subsample_freq': [1,2,4,5,6,7,8]
 }

import random

param_catboost={
    "learning_rate":random.randrange(start=0.5,stop=0.1,step=0.05,_int=float),
    "depth":random.randrange(1,5),
    "n_estimators":random.randrange(100,1000,50),
    "loss_function":["CrossEntropy"],
    "reg_lambda":random.randrange(0.1,0.5,0.05),
    "eval_metric":["Accuracy"],
    "verbose":[False],
    "task_type":["GPU"]
 }

ValueError: ignored

In [0]:
scale = MinMaxScaler()
x = scale.fit_transform(x)
x_train,x_test , y_train,y_test = train_test_split(x,y,test_size=0.2)
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV

random = RandomizedSearchCV(cat,param_distributions=param_catboost,cv=10,verbose=True,n_jobs=2)
random.fit(x_train,y_train)
random.score(x_test,y_test)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 13.4min
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed: 31.3min finished


0.8115163147792707

In [0]:
random.best_params_

{'depth': 1,
 'eval_metric': 'Accuracy',
 'learning_rate': 0.5,
 'loss_function': 'CrossEntropy',
 'n_estimators': 900,
 'reg_lambda': 0.1,
 'task_type': 'GPU',
 'verbose': False}

In [0]:
import numpy as np

test=testing.drop(["id"],axis=1)
test=scale.transform(test)
al = random.predict(test)
np.unique(random.predict(test),return_counts=True)

(array([0., 1.]), array([3869, 2644]))

In [0]:
pd.DataFrame(data=list(zip(testing.id,al)),columns=["id","netgain"],index=None).to_csv("submission.csv",index=None)

In [0]:
testing

,id,realtionship_status,industry,genre,targeted_sex,average_runtime(minutes_per_week),airtime,airlocation,ratings,expensive,money_back_guarantee
0,1,Widowed,Auto,Comedy,Female,10,Daytime,United-States,0.027465,Low,No
1,4,Married-civ-spouse,Pharma,Comedy,Male,40,Morning,United-States,0.056262,High,Yes
2,5,Divorced,Entertainment,Comedy,Female,50,Morning,United-States,0.027465,Low,No
3,9,Married-civ-spouse,Pharma,Infomercial,Male,40,Primetime,United-States,0.027465,Low,No
4,10,Married-civ-spouse,Pharma,Comedy,Male,40,Primetime,United-States,0.027465,Low,Yes
...,...,...,...,...,...,...,...,...,...,...,...
6508,32538,Never-married,Political,Comedy,Male,40,Morning,United-States,0.027465,High,No
6509,32542,Never-married,Political,Comedy,Male,80,Morning,United-States,0.027465,Low,No
6510,32549,Never-married,Political,Comedy,Male,40,Morning,United-States,0.027465,High,No
6511,32558,Married-civ-spouse,Pharma,Comedy,Male,40,Primetime,United-States,0.027465,High,Yes


In [0]:
import numpy as np
np.unique(model.predict(testing.iloc[:,1:].values),return_counts=True)
# testing.iloc[0]

NameError: ignored

In [0]:
import numpy as np
def out(one,two,three,row):
  l = []
  l.append(one.predict(row)[0])
  l.append(two.predict(row)[0])
  l.append(three.predict(row)[0])
  val,count = np.unique(l,return_counts=True)
  return val[np.argmax(count)]

al =[]

for i,row in testing.iloc[:,1:].iterrows():
  # row = scale.transform([row])
  # print(row)
  al.append(cat.predict([row])[0])

np.unique(al,return_counts=True)

(array([0., 1.]), array([5468, 1045]))

In [0]:
pd.DataFrame(data=list(zip(testing.id,al)),columns=["id","netgain"],index=None).to_csv("submission.csv",index=None)

#Neural Network

In [0]:
from keras.layers import Dropout,Dense,Input , Conv1D , MaxPool1D , Activation , BatchNormalization , RepeatVector,Flatten
from keras.models import Model
from keras.utils import to_categorical

In [0]:
x = data.drop(["id","netgain"],axis=1)
y= data[["netgain"]]

In [0]:
y=to_categorical(y)

In [0]:
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
scale.fit(x)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
neural_x_train = scale.transform(x_train)
neural_x_test = scale.transform(x_test)

In [0]:
input = Input(shape=(10,))
output = RepeatVector(10)(input)
output = Conv1D(filters=2,kernel_size=2)(output)
output = MaxPool1D()(output)
output = Conv1D(filters=2,kernel_size=2)(output)
output = MaxPool1D()(output)
output = Flatten()(output)
output = Dense(64)(output)
batch = BatchNormalization()(output)
output = Dense(32)(batch)
batch = BatchNormalization()(output)
output = Dense(16)(output)
output = Dropout(0.2)(output)
batch = BatchNormalization()(output)
output = Dense(4)(output)
batch = BatchNormalization()(output)
output = Dense(2,activation="sigmoid")(output)

In [0]:
model = Model(input = input , output=output)
model.summary()


Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 10)                0         
_________________________________________________________________
repeat_vector_10 (RepeatVect (None, 10, 10)            0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 9, 2)              42        
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 4, 2)              0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 3, 2)              10        
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 1, 2)              0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 2)                 0  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [0]:
model.compile(loss="categorical_crossentropy",metrics=["acc"],optimizer="adam")
model.fit(x,y,epochs=25,verbose=1,batch_size=32)

Epoch 1/25
26048/26048 [==============================] - 7s 277us/step - loss: 0.4884 - acc: 0.7557
Epoch 2/25
26048/26048 [==============================] - 6s 213us/step - loss: 0.4629 - acc: 0.7683
Epoch 3/25
26048/26048 [==============================] - 6s 218us/step - loss: 0.4539 - acc: 0.7736
Epoch 4/25
26048/26048 [==============================] - 5s 210us/step - loss: 0.4514 - acc: 0.7786
Epoch 5/25
26048/26048 [==============================] - 5s 210us/step - loss: 0.4439 - acc: 0.7836
Epoch 6/25
26048/26048 [==============================] - 5s 207us/step - loss: 0.4395 - acc: 0.7869
Epoch 7/25
26048/26048 [==============================] - 5s 209us/step - loss: 0.4333 - acc: 0.7899
Epoch 8/25
26048/26048 [==============================] - 5s 207us/step - loss: 0.4325 - acc: 0.7914
Epoch 9/25
26048/26048 [==============================] - 5s 210us/step - loss: 0.4333 - acc: 0.7896
Epoch 10/25
26048/26048 [==============================] - 6s 212us/step - loss: 0.4322 - a

In [0]:
_,accuracy = model.evaluate(neural_x_test,y_test)
_,accuracy  

5210/5210 [==============================] - 1s 165us/step


(1.2328863332916815, 0.7702495200849083)

In [0]:
model.predict(testing.iloc[:10,1:])

array([[0.98055536, 0.00588229],
       [0.78692436, 0.2334266 ],
       [0.9654776 , 0.03533936],
       [0.8898759 , 0.25118876],
       [0.84364426, 0.25463128],
       [0.84118736, 0.03727129],
       [0.7339128 , 0.07850137],
       [0.9133587 , 0.06512684],
       [0.94811225, 0.04615781],
       [0.9652698 , 0.04894671]], dtype=float32)

In [0]:
al =[]

for i,row in testing.iloc[:,1:].iterrows():
  row = scale.transform([row])
  al.append(model.predict(row)[0])

import numpy as np
np.unique(al,return_counts=True)

(array([0.0000000e+00, 1.9101924e-04, 1.0775853e-03, ..., 9.9276102e-01,
        9.9302894e-01, 9.9329668e-01], dtype=float32),
 array([38,  1,  1, ...,  1,  1,  1]))

In [0]:
import numpy as np

np.unique(np.argmax(model.predict(testing.iloc[:,1:].values),axis=1),return_counts=True)

(array([0, 1]), array([6292,  221]))

In [0]:
np.unique(np.argmax(al,axis=1),return_counts=True)

(array([0, 1]), array([6465,   48]))

In [0]:
al = np.argmax(al,axis=1)

In [0]:
pd.DataFrame(data=list(zip(testing.id,al)),columns=["id","netgain"],index=None).to_csv("submission.csv",index=None)